In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

#display(dbutils.fs.ls("/FileStore/tables/"))
df = spark.read.csv("dbfs:/FileStore/tables/2011_12_07.csv", header=True, inferSchema=True)
#display(df)

# ZADANIE 1
# fill - fill i fillna zwaraca to samo, chodzi o zastąpienie wartosci null
df_fill = df.na.fill({"CustomerID": 0})
#display(df_fill)


# explode - rozbijanie tablicy na soobne kolumny - TO POLECENIE NIE PRZEJDZIE! - nie mam wsrod tych danych kolumny typu array, np. : skills =  [php, R]
#df_exploded = df.withColumn("cou", explode("Description")).drop("Description")
#display(df_exploded)


# drop - usniecie wierszy z null
df_drop = df.dropna(subset=["Description"])
#display(df_drop)


# regexp_replace - służy do zamiany fragmentów tekstu w kolumnach DataFrame na podstawie wyrażeń regularnych (regex)
df_regex = df.withColumn("col_regex", regexp_replace(col("Country"), r"\w{3}$", "XXX"))
#display(df_regex)


# regexp_extract - służy do wyciągania fragmentów tekstu z kolumny na podstawie wyrażenia regularnego (regex
df_reg2 = df.withColumn("invoice_type",regexp_extract(col("InvoiceNo"), r"^([A-Z])", 1))  # Pierwsza litera (S, C, itp.)
#display(df_reg2)


# ifnull - 
df_ifnull = df.withColumn("customerid_filled", expr("ifnull(CustomerID, 0)"))  # wstaw 0 gdy CustomerID jest null)
#display(df_ifnull)


#nullif - zwraca null gdy wartosci sa rowne
df_nullIf = df.withColumn("uk_null", expr("nullIf(Country, 'United Kingdom')"))  # NULL, jeśli kraj to uk
#display(df_nullIf)


# replace - zamiana okreslonych wartosci na inne
df_replace = df.replace("Germany", "X")
#display(df_replace)


# array_contains - służy do sprawdzania, czy tablica (kolumna typu Array) zawiera określoną wartość. Zwraca True, jeśli wartość istnieje w tablicy, lub False, jeśli nie.
#stworzymy najpiewr tablice
df_with_array = df.withColumn("product_info_array", array(col("StockCode"), col("Description")))
#display(df_with_array.select("product_info_array"))

df_with_flag = df_with_array.withColumn("contains_check", array_contains(col("product_info_array"), "check"))
#display(df_with_flag.filter(col("contains_check") == True))


# PRZYKLAD B
c_stats = df.groupBy("Country").agg(
    sum("Quantity").alias("total_quantity"),       #suma zamowien
    avg("UnitPrice").alias("avg_price"),          #srednia cenka produktu
    count("*").alias("order_count")               #liczba zamowien
).orderBy("total_quantity", ascending=False)




In [0]:

## ZADNIE 2
# funckja UDF - mechanizm ktory pozwala na stworzenie wlasnych, niestandardowych funkcji 
#dla double; pandas
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import DoubleType
import pandas as pd

@pandas_udf(DoubleType())
def calculate_discount(price: pd.Series, rating: pd.Series) -> pd.Series:
    """
    oblicza zniezke na produkt procentowo
    - ceny (im wyższa cena, tym większa zniżka)
    - oceny (im wyższa ocena, tym większa zniżka)
    
    znizka od 0 do 20%
    - 5% podstawowej zniżki
    - dodatkowo do 10% za cenę (1% za każde 100 jednostek ceny)
    - dodatkowo do 5% za ocenę (1% za każdy punkt oceny powyżej 3)
    """
    base_discount = 5
    price_discount = (price / 100).clip(upper=10)
    rating_discount = ((rating - 3) * 1).clip(lower=0, upper=5)
    
    total_discount = base_discount + price_discount + rating_discount
    return total_discount.clip(upper=20)

In [0]:
#ZADANIE 2
# UDF - dla stringow
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

@udf(StringType())
def format_product_name(name: str, category: str) -> str:
    """
    formatowanie nazwy i kategorii produktu
    - minus biale znaki na poczatku i koncu
    - zmiana na male litery
    - Dodaje kategorię w nawiasach jeśli nie jest pusta
    - wielokrotne spacje na pojedyncze
    """
    if name is None:
        return None
        
    name_clean = ' '.join(name.strip().lower().split())
    category_clean = ' '.join(category.strip().lower().split()) if category else ''
    
    if category_clean:
        return f"{name_clean} ({category_clean})"
    return name_clean